In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy as sa

import gspread
from gspread_dataframe import get_as_dataframe

import pandas as pd

# Load service account credentials
gc = gspread.service_account(filename="service_account.json")

# Open the spreadsheet by ID
sh = gc.open_by_key("1jdSSr3n6aJDjbsl0JnNekCVrfoyCM4D32PLYYPDYWMY")

# Select the sheet by name
worksheet = sh.worksheet("stgData")  # exact tab name

# Convert to pandas DataFrame
df = get_as_dataframe(
    worksheet,
    evaluate_formulas=True
)


# Preview
df.tail()


,date,transaction,amount
184,2026-02-06,Remainder,100.0
185,2026-02-20,Remainder,1005.0
186,2026-03-06,Remainder,1150.0
187,2026-03-20,Remainder,1145.0
188,2026-04-03,Remainder,2200.0


In [12]:
transaction_types = df["transaction"].unique()
print(transaction_types)


['Headspace' 'SUNY' 'Duet' 'York' 'Total Income' 'Mortgage' 'Duke'
 'Water/Trash' 'Groceries' 'T-Mobile' 'Car Insurance' 'TransAmerica'
 'IRA Contributions' 'Allowances' 'Gym' 'Windows (BofA)' 'Cleaning'
 'Kia (CapitalOne)' 'Total Bills' 'Remainder']


In [13]:
remove_values = ["Total Income", "Total Bills", "Remainder"]
stg_df = df[~df["transaction"].isin(remove_values)]
stg_df

,date,transaction,amount
0,2026-04-17,Headspace,500.0
1,2026-05-01,Headspace,500.0
2,2026-05-15,Headspace,500.0
3,2026-05-29,Headspace,500.0
4,2025-12-12,Headspace,1000.0
...,...,...,...
158,2026-05-29,Kia (CapitalOne),-400.0
159,2025-12-26,Kia (CapitalOne),-400.0
160,2026-01-23,Kia (CapitalOne),-400.0
161,2026-02-20,Kia (CapitalOne),-400.0


In [18]:
stg_df["type"] = np.where(stg_df["amount"] < 0, "bill", "income")

stg_df["date"] = pd.to_datetime(stg_df["date"])
stg_df["month"] = stg_df["date"].dt.month
stg_df["year"] = stg_df["date"].dt.year

stg_df


/var/folders/d_/n324z2xs76d63sdx1pngjl5h0000gn/T/ipykernel_11702/2902724800.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stg_df["type"] = np.where(stg_df["amount"] < 0, "bill", "income")
/var/folders/d_/n324z2xs76d63sdx1pngjl5h0000gn/T/ipykernel_11702/2902724800.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stg_df["date"] = pd.to_datetime(stg_df["date"])
/var/folders/d_/n324z2xs76d63sdx1pngjl5h0000gn/T/ipykernel_11702/2902724800.py:4: SettingWithCopyWarning: 
A value is trying to be set on a co

,date,transaction,amount,type,month,year
0,2026-04-17,Headspace,500.0,income,4,2026
1,2026-05-01,Headspace,500.0,income,5,2026
2,2026-05-15,Headspace,500.0,income,5,2026
3,2026-05-29,Headspace,500.0,income,5,2026
4,2025-12-12,Headspace,1000.0,income,12,2025
...,...,...,...,...,...,...
158,2026-05-29,Kia (CapitalOne),-400.0,bill,5,2026
159,2025-12-26,Kia (CapitalOne),-400.0,bill,12,2025
160,2026-01-23,Kia (CapitalOne),-400.0,bill,1,2026
161,2026-02-20,Kia (CapitalOne),-400.0,bill,2,2026


In [ ]:
from gspread_dataframe import set_with_dataframe


gc = gspread.service_account(filename="service_account.json")

# Open the Google Sheet by ID
sh = gc.open_by_key("1jdSSr3n6aJDjbsl0JnNekCVrfoyCM4D32PLYYPDYWMY")

# Create a new worksheet/tab (or overwrite if it exists)
worksheet_title = "processedData"

# Delete old worksheet if exists
try:
    sh.del_worksheet(sh.worksheet(worksheet_title))
except gspread.WorksheetNotFound:
    pass

# Create new worksheet
worksheet = sh.add_worksheet(title=worksheet_title, rows=str(len(stg_df)+1), cols=str(len(stg_df.columns)))

# Write the DataFrame to the new worksheet
set_with_dataframe(worksheet, stg_df)